In [15]:
from elasticsearch import Elasticsearch
client = Elasticsearch()

In [82]:
query = {
    'bool': {
        'must':{'match': {'response_code':403}},
        'filter':{
            'range': {
                'start_time': {
                    'gte': '2022-05-15'
                }
            }
        }
    }
}
response = client.search(
    index="envoy-k8s",
    query=query,
    sort={"start_time": {"order": "desc"}},
    size = 1000
)

In [80]:
response['hits']

{'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}

In [83]:
ips = {}
for hit in response['hits']['hits']:
  source = hit['_source']
  x_forwarded_for = source['x_forwarded_for']
  if source['response_code'] == '403':
    count = ips.get(x_forwarded_for, 0)
    count = count + 1
    ips[x_forwarded_for] = count

print(f"total: {response['hits']['total']['value']}")
for ips, count in ips.items():
  print(f"{ips}: {count}")

total: 40
10.244.1.1: 21
10.244.1.1,10.244.1.1: 4
56.5.6.7, 72.9.5.6, 98.1.2.3,10.244.1.1: 15


In [17]:
client.info()

{'name': 'elasticsearch',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'olYMaE7kRSmPJJ5kOuHg4g',
 'version': {'number': '7.9.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '083627f112ba94dffc1232e8b42b73492789ef91',
  'build_date': '2020-09-01T21:22:21.964974Z',
  'build_snapshot': False,
  'lucene_version': '8.6.2',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}